In [0]:
#bronze,silver&gold
%sql
create table bronze_eps01
(date STRING, stock_symbol STRING, analyst INT, estimated_eps DOUBLE)
using DELTA
TBLPROPERTIES (delta.enableChangeDataFeed-true);
create table silver_eps01
(date STRING,stock_symbol STRING, analyst INT, estimated_eps DOUBLE)
using DELTA
TBLPROPERTIES (delta.enableChangeDataFeed=true);
create table gold_eps01
(date STRING, stock_symbol STRING, consensus_eps DOUBLE)
using DELTA
TBLPROPERTIES(delta.enableChangeDataFeed=true);

In [0]:
%sql
create table bronze_table1
(data string, stock_symbol string, analyst int, estimated_eps double)
using delta
tblproperties (delta.enableChangeDataFeed=true);

In [0]:
#inserting the data in python notebook
data=[('3/1/2021','a',1,2.2),('3/1/2021','b',2,2.0),('3/1/2021','b',1,1.3),('3/1/2021','c',2,1.2),('3/1/2021','c',1,2.6),('3/1/2021','a',1,
2.2)]
columns=['date', 'stock_symbol','analyst','estimated_eps']
df=spark.createDataFrame(data=data,schema=columns)
df.createOrReplaceTempView("bronze_table1_march_dataset1")

In [0]:
%sql
INSERT INTO bronze_table1 TABLE bronze_table1_march_dataset1

num_affected_rows,num_inserted_rows
6,6


In [0]:
%sql
select * from table_changes ("bronze_table1", 1)

data,stock_symbol,analyst,estimated_eps,_change_type,_commit_version,_commit_timestamp
3/1/2021,b,2,2.0,insert,1,2023-05-22T07:08:58.000+0000
3/1/2021,b,1,1.3,insert,1,2023-05-22T07:08:58.000+0000
3/1/2021,c,1,2.6,insert,1,2023-05-22T07:08:58.000+0000
3/1/2021,a,1,2.2,insert,1,2023-05-22T07:08:58.000+0000
3/1/2021,a,1,2.2,insert,1,2023-05-22T07:08:58.000+0000
3/1/2021,c,2,1.2,insert,1,2023-05-22T07:08:58.000+0000


In [0]:
%sql
create table silver_table2
(data STRING,stock_symbol STRING, analyst INT, estimated_eps DOUBLE)
using DELTA
TBLPROPERTIES (delta.enableChangeDataFeed=true);

In [0]:
%sql
INSERT INTO silver_table2
SELECT data, stock_symbol, analyst, estimated_eps
FROM table_changes('bronze_table1', 1)

num_affected_rows,num_inserted_rows
6,6


In [0]:
%sql
SELECT * FROM table_changes('silver_table2', 1)

data,stock_symbol,analyst,estimated_eps,_change_type,_commit_version,_commit_timestamp
3/1/2021,b,2,2.0,insert,1,2023-05-22T07:32:11.000+0000
3/1/2021,b,1,1.3,insert,1,2023-05-22T07:32:11.000+0000
3/1/2021,c,1,2.6,insert,1,2023-05-22T07:32:11.000+0000
3/1/2021,a,1,2.2,insert,1,2023-05-22T07:32:11.000+0000
3/1/2021,a,1,2.2,insert,1,2023-05-22T07:32:11.000+0000
3/1/2021,c,2,1.2,insert,1,2023-05-22T07:32:11.000+0000


In [0]:
%sql
create table gold_table3
(date STRING, stock_symbol STRING, consensus_eps DOUBLE)
using DELTA
TBLPROPERTIES(delta.enableChangeDataFeed=true);

In [0]:
%sql
INSERT into gold_table3
select data, stock_symbol,AVG (estimated_eps) as consensus_eps from silver_table2 group by data, stock_symbol;

num_affected_rows,num_inserted_rows
3,3


In [0]:
%sql
SELECT * FROM table_changes('gold_table3', 1)

date,stock_symbol,consensus_eps,_change_type,_commit_version,_commit_timestamp
3/1/2021,b,1.65,insert,1,2023-05-22T09:02:25.000+0000
3/1/2021,c,1.9,insert,1,2023-05-22T09:02:25.000+0000
3/1/2021,a,2.2,insert,1,2023-05-22T09:02:25.000+0000
